**Fairness Definition Selector** </br></br>
The Fairness Definition Selector module is designed to help users recognize different definitions of fairness and choose the most appropriate approach for specific contexts. Using a visual 2x2 grid interface with integrated confusion matrix analysis, users compare Demographic Parity, Equal Opportunity, Equal Odds, and Fairness Through Unawareness approaches. Users see exactly how each definition would change algorithm performance metrics for different groups, understanding the mathematical reality and trade-offs of each fairness choice. This develops the advanced skill of contextual fairness reasoning - knowing which fairness definition to apply based on domain requirements, stakeholder needs, and practical constraints.

In [ ]:
!pip install ipywidgets

In [ ]:
import random
import time
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Layout, GridBox

In [ ]:
scenarios = [
    {
        "title": "E-commerce Pricing Algorithm Disparity",
        "description": "Your e-commerce platform's dynamic pricing algorithm gives discounts to customers. Analysis reveals disparities between customer groups.",
        "context": "Customers from minority-majority zip codes receive discounts 40% of the time, while customers from other areas receive discounts 60% of the time.",
        "current_performance": {
            "Group A (Minority Areas)": {
                "total_customers": 1000,
                "deserving_discount": 300,  # True condition
                "not_deserving_discount": 700,
                "tp": 120,  # Correctly gave discount to deserving customers
                "fp": 280,  # Incorrectly gave discount to non-deserving customers
                "tn": 420,  # Correctly didn't give discount to non-deserving customers
                "fn": 180   # Incorrectly didn't give discount to deserving customers
            },
            "Group B (Other Areas)": {
                "total_customers": 1000,
                "deserving_discount": 300,  # Same base rate
                "not_deserving_discount": 700,
                "tp": 240,  # Much better at giving discounts to deserving customers
                "fp": 360,  # More liberal with discounts overall
                "tn": 340,  # Less conservative
                "fn": 60    # Much fewer missed deserving customers
            }
        }
    }
]

fairness_definitions = {
    "Demographic Parity": {
        "short_name": "Demographic Parity",
        "description": "Equal discount rates across groups",
        "formula": "P(discount|Group A) = P(discount|Group B)",
        "color": "#E74C3C",
        "optimizes": "Equal overall outcomes"
    },
    "Equal Opportunity": {
        "short_name": "Equal Opportunity",
        "description": "Equal True Positive Rates",
        "formula": "TPR_A = TPR_B",
        "color": "#3498DB",
        "optimizes": "Fair treatment of deserving customers"
    },
    "Equal Odds": {
        "short_name": "Equal Odds",
        "description": "Equal TPR and FPR across groups",
        "formula": "TPR_A = TPR_B AND FPR_A = FPR_B",
        "color": "#9B59B6",
        "optimizes": "Complete prediction parity"
    },
    "Fairness Through Unawareness": {
        "short_name": "FT Unawareness",
        "description": "Ignore group membership entirely",
        "formula": "No group-based features used",
        "color": "#F39C12",
        "optimizes": "Group-blind decisions"
    }
}

In [ ]:
class FairnessGridSelector:
    def __init__(self, scenarios_list, definitions_dict):
        self.scenarios = scenarios_list
        self.definitions = definitions_dict
        self.current_scenario = 0
        self.selected_definition = None
        self.selections_made = []

        # Create main widgets
        self.scenario_display = HTML(value="<h3>Fairness Definition Grid Selector</h3>")
        self.current_metrics_display = HTML(value="")
        self.grid_display = HTML(value="")
        self.selection_details = HTML(value="")
        self.confirm_button = Button(
            description='Choose This Approach',
            button_style='success',
            layout=Layout(width='200px', height='40px', margin='10px'),
            disabled=True
        )
        self.next_button = Button(
            description='Next Scenario →',
            button_style='',
            layout=Layout(width='150px', height='30px', margin='10px')
        )
        self.progress_display = HTML(value="")

        # Create individual selection buttons
        self.selection_buttons = {}
        for def_name in self.definitions.keys():
            btn = Button(
                description=f'Select {def_name}',
                layout=Layout(width='250px', height='30px', margin='5px')
            )
            btn.on_click(lambda b, name=def_name: self.select_definition(name))
            self.selection_buttons[def_name] = btn

        # Button clicks
        self.confirm_button.on_click(lambda b: self.confirm_selection())
        self.next_button.on_click(lambda b: self.next_scenario())

        # Layout
        button_box = HBox([self.confirm_button], layout=Layout(justify_content='center'))
        next_box = HBox([self.next_button], layout=Layout(justify_content='center'))

        selection_button_box = HBox(
            list(self.selection_buttons.values()),
            layout=Layout(justify_content='center', flex_wrap='wrap')
        )

        self.widget = VBox([
            self.scenario_display,
            self.current_metrics_display,
            HTML(value="<h4 style='text-align: center; color: #2C3E50;'>Select a Fairness Approach:</h4>"),
            self.grid_display,
            selection_button_box,
            self.selection_details,
            button_box,
            next_box,
            self.progress_display
        ])

    def start_selection(self):
        self.current_scenario = 0
        self.selections_made = []
        self.show_scenario()
        return self.widget

    def show_scenario(self):
        if self.current_scenario >= len(self.scenarios):
            self.show_final_analysis()
            return

        scenario = self.scenarios[self.current_scenario]
        self.selected_definition = None
        self.confirm_button.disabled = True
        self.next_button.layout.display = 'none'

        self.scenario_display.value = f"""
        <div style='padding: 20px; background: #f9f9f9; border-radius: 10px; margin: 10px 0; text-align: center;'>
            <h3>Scenario: {scenario['title']}</h3>
            <p style='font-size: 16px; line-height: 1.5;'><strong>{scenario['description']}</strong></p>
            <p style='color: #666;'>{scenario['context']}</p>
        </div>
        """

        self.show_current_metrics(scenario)
        self.create_fairness_grid()
        self.selection_details.value = ""
        self.update_progress()

    def show_current_metrics(self, scenario):
        perf = scenario['current_performance']
        group_a = perf['Group A (Minority Areas)']
        group_b = perf['Group B (Other Areas)']

        a_tpr = group_a['tp'] / (group_a['tp'] + group_a['fn'])
        a_fpr = group_a['fp'] / (group_a['fp'] + group_a['tn'])
        a_discount_rate = (group_a['tp'] + group_a['fp']) / group_a['total_customers']

        b_tpr = group_b['tp'] / (group_b['tp'] + group_b['fn'])
        b_fpr = group_b['fp'] / (group_b['fp'] + group_b['tn'])
        b_discount_rate = (group_b['tp'] + group_b['fp']) / group_b['total_customers']

        self.current_metrics_display.value = f"""
        <div style='border: 2px solid #95A5A6; padding: 15px; border-radius: 8px; margin: 10px 0; background: #ECF0F1;'>
            <h4 style='color: #2C3E50; text-align: center;'>📊 Current Algorithm Performance</h4>
            <div style='display: flex; justify-content: space-around; margin-top: 15px;'>
                <div style='text-align: center;'>
                    <h5>Group A (Minority Areas)</h5>
                    <div style='background: white; padding: 10px; border-radius: 5px; margin: 5px;'>
                        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 5px; font-size: 12px;'>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TP: {group_a['tp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FP: {group_a['fp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FN: {group_a['fn']}</div>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TN: {group_a['tn']}</div>
                        </div>
                    </div>
                    <p><strong>Discount Rate:</strong> {a_discount_rate:.1%}</p>
                    <p><strong>TPR:</strong> {a_tpr:.1%} | <strong>FPR:</strong> {a_fpr:.1%}</p>
                </div>
                <div style='text-align: center;'>
                    <h5>Group B (Other Areas)</h5>
                    <div style='background: white; padding: 10px; border-radius: 5px; margin: 5px;'>
                        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 5px; font-size: 12px;'>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TP: {group_b['tp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FP: {group_b['fp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FN: {group_b['fn']}</div>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TN: {group_b['tn']}</div>
                        </div>
                    </div>
                    <p><strong>Discount Rate:</strong> {b_discount_rate:.1%}</p>
                    <p><strong>TPR:</strong> {b_tpr:.1%} | <strong>FPR:</strong> {b_fpr:.1%}</p>
                </div>
            </div>
        </div>
        """

    def create_fairness_grid(self):
        definition_items = list(self.definitions.items())

        grid_html = """
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px; max-width: 800px; margin: 0 auto;'>
        """

        for def_name, def_info in definition_items:
            selected_style = ""
            if self.selected_definition == def_name:
                selected_style = "border-width: 5px; background: #F8F9FA;"

            grid_html += f"""
            <div style='
                border: 3px solid {def_info['color']};
                border-radius: 10px;
                padding: 20px;
                background: white;
                text-align: center;
                min-height: 150px;
                {selected_style}
            '>
                <h4 style='color: {def_info['color']}; margin: 0 0 10px 0;'>{def_info['short_name']}</h4>
                <p style='margin: 8px 0; font-size: 14px;'>{def_info['description']}</p>
                <p style='margin: 8px 0; font-size: 12px; color: #666;'><strong>Formula:</strong> {def_info['formula']}</p>
                <p style='margin: 8px 0; font-size: 12px; font-style: italic; color: {def_info['color']};'>Optimizes: {def_info['optimizes']}</p>
                {'<p style="margin: 8px 0; font-size: 12px; color: #27AE60; font-weight: bold;">✓ SELECTED</p>' if self.selected_definition == def_name else ''}
            </div>
            """

        grid_html += "</div>"
        self.grid_display.value = grid_html

    def select_definition(self, def_name):
        self.selected_definition = def_name
        self.confirm_button.disabled = False
        scenario = self.scenarios[self.current_scenario]
        self.show_selection_details(def_name, scenario)
        self.create_fairness_grid()

    def show_selection_details(self, def_name, scenario):
        adjusted_metrics = self.calculate_adjusted_metrics(def_name, scenario)
        def_info = self.definitions[def_name]

        self.selection_details.value = f"""
        <div style='border: 3px solid {def_info['color']}; padding: 20px; border-radius: 10px; margin: 15px 0; background: #FAFAFA;'>
            <h4 style='color: {def_info['color']}; text-align: center;'>📋 Impact of {def_name}</h4>

            <div style='display: flex; justify-content: space-around; margin: 15px 0;'>
                <div style='text-align: center;'>
                    <h5>Group A After Adjustment</h5>
                    <div style='background: white; padding: 10px; border-radius: 5px; margin: 5px;'>
                        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 5px; font-size: 12px;'>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TP: {adjusted_metrics['group_a']['tp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FP: {adjusted_metrics['group_a']['fp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FN: {adjusted_metrics['group_a']['fn']}</div>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TN: {adjusted_metrics['group_a']['tn']}</div>
                        </div>
                    </div>
                    <p><strong>TPR:</strong> {adjusted_metrics['group_a']['tpr']:.1%} | <strong>FPR:</strong> {adjusted_metrics['group_a']['fpr']:.1%}</p>
                    <p><strong>Discount Rate:</strong> {adjusted_metrics['group_a']['discount_rate']:.1%}</p>
                </div>

                <div style='text-align: center;'>
                    <h5>Group B After Adjustment</h5>
                    <div style='background: white; padding: 10px; border-radius: 5px; margin: 5px;'>
                        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 5px; font-size: 12px;'>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TP: {adjusted_metrics['group_b']['tp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FP: {adjusted_metrics['group_b']['fp']}</div>
                            <div style='background: #E74C3C; padding: 5px; color: white;'>FN: {adjusted_metrics['group_b']['fn']}</div>
                            <div style='background: #2ECC71; padding: 5px; color: white;'>TN: {adjusted_metrics['group_b']['tn']}</div>
                        </div>
                    </div>
                    <p><strong>TPR:</strong> {adjusted_metrics['group_b']['tpr']:.1%} | <strong>FPR:</strong> {adjusted_metrics['group_b']['fpr']:.1%}</p>
                    <p><strong>Discount Rate:</strong> {adjusted_metrics['group_b']['discount_rate']:.1%}</p>
                </div>
            </div>

            <div style='background: #FFFFCC; padding: 10px; border-radius: 5px; margin-top: 15px;'>
                <p><strong>🎯 What This Achieves:</strong> {adjusted_metrics['achievement']}</p>
                <p><strong>⚖️ Trade-off:</strong> {adjusted_metrics['tradeoff']}</p>
            </div>
        </div>
        """

    def calculate_adjusted_metrics(self, def_name, scenario):
        perf = scenario['current_performance']
        group_a = perf['Group A (Minority Areas)'].copy()
        group_b = perf['Group B (Other Areas)'].copy()

        if def_name == "Demographic Parity":
            group_a.update({'tp': 150, 'fp': 350, 'fn': 150, 'tn': 350})
            group_b.update({'tp': 150, 'fp': 350, 'fn': 150, 'tn': 350})
            achievement = "Equal 50% discount rates across both groups"
            tradeoff = "Some deserving customers in Group A still missed, some undeserving in Group B get discounts"

        elif def_name == "Equal Opportunity":
            group_a.update({'tp': 240, 'fp': 280, 'fn': 60, 'tn': 420})
            group_b.update({'tp': 240, 'fp': 360, 'fn': 60, 'tn': 340})
            achievement = "Equal 80% True Positive Rate - deserving customers equally identified"
            tradeoff = "Overall discount rates still differ between groups"

        elif def_name == "Equal Odds":
            group_a.update({'tp': 210, 'fp': 315, 'fn': 90, 'tn': 385})
            group_b.update({'tp': 210, 'fp': 315, 'fn': 90, 'tn': 385})
            achievement = "Equal TPR (70%) and FPR (45%) across both groups"
            tradeoff = "Complex calibration required, some accuracy loss"

        else:  # Fairness Through Unawareness
            group_a.update({'tp': 140, 'fp': 260, 'fn': 160, 'tn': 440})
            group_b.update({'tp': 220, 'fp': 380, 'fn': 80, 'tn': 320})
            achievement = "Algorithm ignores group membership completely"
            tradeoff = "Geographic and other proxy variables may still cause disparities"

        def calc_rates(group):
            tpr = group['tp'] / (group['tp'] + group['fn'])
            fpr = group['fp'] / (group['fp'] + group['tn'])
            discount_rate = (group['tp'] + group['fp']) / 1000
            return {**group, 'tpr': tpr, 'fpr': fpr, 'discount_rate': discount_rate}

        return {
            'group_a': calc_rates(group_a),
            'group_b': calc_rates(group_b),
            'achievement': achievement,
            'tradeoff': tradeoff
        }

    def confirm_selection(self):
        if not self.selected_definition:
            return

        scenario = self.scenarios[self.current_scenario]
        self.selections_made.append({
            'scenario': scenario['title'],
            'selected_definition': self.selected_definition
        })

        self.selection_details.value += """
        <div style='background: #D5EDDA; border: 2px solid #C3E6CB; padding: 15px; border-radius: 8px; margin: 15px 0; text-align: center;'>
            <h4 style='color: #155724; margin: 0;'>✅ Selection Confirmed!</h4>
            <p style='margin: 10px 0;'>You have chosen this fairness approach for implementation.</p>
        </div>
        """

        self.confirm_button.disabled = True
        self.next_button.layout.display = 'block'

    def next_scenario(self):
        self.current_scenario += 1
        self.show_scenario()

    def update_progress(self):
        total_scenarios = len(self.scenarios)

        self.progress_display.value = f"""
        <div style='background: #e8f4fd; padding: 10px; border-radius: 5px; text-align: center;'>
            <strong>Progress:</strong> Scenario {self.current_scenario + 1}/{total_scenarios} |
            <strong>Selections Made:</strong> {len(self.selections_made)}
        </div>
        """

    def show_final_analysis(self):
        self.scenario_display.value = """
        <div style='border: 3px solid #4CAF50; padding: 25px; border-radius: 15px; background: #f8fff8; text-align: center;'>
            <h2 style='color: #4CAF50;'>Fairness Definition Selection Complete! 🎯</h2>

            <h3>Developed Skills</h3>
            <ul style='text-align: left; max-width: 600px; margin: 0 auto;'>
                <li><strong>Visual Comparison:</strong> Evaluated fairness definitions side-by-side</li>
                <li><strong>Confusion Matrix Analysis:</strong> Understood how each definition affects TP, FP, TN, FN</li>
                <li><strong>Trade-off Assessment:</strong> Weighed benefits vs. costs of each approach</li>
                <li><strong>Contextual Selection:</strong> Chose appropriate definitions for specific situations</li>
            </ul>

            <h4>Key Insight: Mathematical Reality of Fairness</h4>
            <p>You've experienced how different fairness definitions create different confusion matrices and outcomes.
            This mathematical understanding is crucial for implementing fairness effectively in real systems.</p>
        </div>
        """

        self.current_metrics_display.value = ""
        self.grid_display.value = ""
        self.selection_details.value = ""
        self.confirm_button.layout.display = 'none'
        self.next_button.layout.display = 'none'

In [ ]:
# Start the Fairness Grid Selector
selector = FairnessGridSelector(scenarios, fairness_definitions)
widget = selector.start_selection()
display(widget)